# ABUNDANCE DERIVATIONS WITH THE DIRECT METHOD

## The chemical composition of IC 2165 (See previous Notebook)

## The chemical composition of giant HII regions NGC 300.

Bresolin et al. 2009, have observed 20 giant H II regions in the spiral galaxy NGC 300.

##### We first import needed libraries

In [ ]:
%matplotlib inline
import pyneb as pn
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, Column

### Compute the electron densities and temperatures from all the available ratios

##### We read the observations into a PyNeb Observation object. Notice that in this file the errors are absolutes.

In [ ]:
obs = pn.Observation('../Data/NGC300.dat', corrected=True, errIsRelative=False)
# the galactocentric distance has been also read and the values are in "obsIntens"
# with a label that does not correspond to a line (warning is issued, 
# it's a normal behaviour).
Rgal = obs.getLine(label='DIST').obsIntens

##### We instanciate the Diagnostic object.

In [ ]:
diags = pn.Diagnostics()

##### We set up the diagnostics according to the available emission lines.

In [ ]:
diags.addDiagsFromObs(obs)

##### We check which diagnostics have been selected.

In [ ]:
diags.diags

##### We check which Atoms have been instantiated in the Diagnostic object. 

In [ ]:
diags.atomDict

In [ ]:
# Clean the pypics files, because we will change the boundaries.
pn.utils.misc.cleanPypicFiles(all_=True)
# we can choose to do the diagnostic plots of the whole set of observations
emisgrids = pn.getEmisGridDict(atomDict=diags.atomDict, n_tem=100, n_den=100, 
                               tem_min=5000., tem_max=15000., 
                               den_min=10., den_max=1e5)

##### We plot all the diagnostics diagrams.

In [ ]:
# We can see which diagnostic to use for Te and for Ne determinations
f, axes = plt.subplots(6, 5, figsize=(20, 20))
for i, obs_name in enumerate(obs.names):
    ax = axes.ravel()[i]
    diags.plot(emisgrids, obs, i_obs=i, ax=ax)
    ax.set_title(obs_name)
f.tight_layout()
f.savefig('diags.pdf')

##### We determine Te and Ne by intersection of 2 diagnostics.

In [ ]:
pn.log_.level = 3
temp_O3, dens_S2a = diags.getCrossTemDen('[OIII] 4363/5007', '[SII] 6731/6716', 
                                         obs=obs)
temp_S3, dens_S2b = diags.getCrossTemDen('[SIII] 6312/9069', '[SII] 6731/6716', 
                                         obs=obs)
temp_N2, dens_S2c = diags.getCrossTemDen('[NII] 5755/6584', '[SII] 6731/6716', 
                                         obs=obs)

In [ ]:
print(temp_O3)

##### In the cases where the density is not defined, we choose to set it to 10 and compute new Te.

In [ ]:
pn.log_.level = 1
mask_nan = np.isnan(dens_S2a)
dens_S2a[mask_nan] = 10.
O3 = pn.Atom('O', 3)
temp_O3[mask_nan] = O3.getTemDen((obs.getLine(label='O3_4363A').corrIntens/
                                  obs.getLine(label='O3_5007A').corrIntens)[mask_nan], 
                                  den=10, wave1=4363, wave2=5007)
mask_nan = np.isnan(dens_S2b)
dens_S2b[mask_nan] = 10.
S3 = pn.Atom('S', 3)
temp_S3[mask_nan] = S3.getTemDen((obs.getLine(label='S3_6312A').corrIntens/
                                  obs.getLine(label='S3_9069A').corrIntens)[mask_nan], 
                                  den=10, wave1=6312, wave2=9069)
mask_nan = np.isnan(dens_S2c)
dens_S2c[mask_nan] = 10.
N2 = pn.Atom('N', 2)
temp_N2[mask_nan] = N2.getTemDen((obs.getLine(label='N2_5755A').corrIntens/
                                  obs.getLine(label='N2_6584A').corrIntens)[mask_nan], 
                                  den=10, wave1=5755, wave2=6583)
    
# Here we adopt an average value of the 3 densities
mean_dens = (dens_S2a + dens_S2b + dens_S2c) / 3

##### We determine electron temperatures from O2 and S2 from the previously determined S2 densities.

In [ ]:
O2 = pn.Atom('O', 2)
temp_O2 = O2.getTemDen((obs.getLine(label='O2_3727A+').corrIntens/
                        obs.getLine(label='O2_7325A+').corrIntens),
                        den = mean_dens, 
                        to_eval = '(L(3726)+L(3729))/(I(4,2)+I(5,2)+I(4,3)+I(5,3))')
S2 = pn.Atom('S', 2)
temp_S2 = S2.getTemDen((obs.getLine(label='S2_4072A+').corrIntens/
                        obs.getLine(label='S2_6716A').corrIntens),
                        den = mean_dens, 
                        to_eval = '(L(4076)+L(4069))/L(6716)')

##### We instantiate a Table object (Table as been previously imported from astropy).

In [ ]:
T = Table()

##### We add the columns with their names into T

In [ ]:
T.add_columns([Column(obs.names, name='NAME'), 
               Column(temp_O2, name='T_O2'),
               Column(temp_S2, name='T_S2'),
               Column(temp_N2, name='T_N2'),
               Column(temp_O3, name='T_O3')])

##### We pretty print the table T.

In [ ]:
print(T)

##### We print the table T in an ascii file

In [ ]:
T.write('temperatures1.ascii', format='ascii.fixed_width')
# writing to a second file with another format
T.write('temperatures2.ascii', format='ascii.fixed_width',
        formats={'NAME':'%5s', 'T_O2': '%7.1f', 'T_N2': '%7.1f', 'T_S2':'%7.1f', 'T_O3':'%7.1f'})

In [ ]:
!cat temperatures2.ascii

##### We print the table T in a latex file

In [ ]:
T.write('temperatures.tex', format='ascii.latex',
        formats={'NAME':'%5s', 'T_O2': '%7.1f', 'T_N2': '%7.1f', 'T_S2':'%7.1f', 'T_O3':'%7.1f'})
# test anither format
T.write('temperatures2.tex', format='ascii.aastex',
        formats={'NAME':'%5s', 'T_O2': '%7.1f', 'T_N2': '%7.1f', 'T_S2':'%7.1f', 'T_O3':'%7.1f'})

In [ ]:
!cat temperatures.tex

### Use Bresolin’s et al policy to derive the electron temperature in the low- and high-excitation zones of each object.

##### We use a copy of the arrays, as we will have to modify them and don't want to touch the original data.

In [ ]:
tS3 = temp_S3.copy()
tO3 = temp_O3.copy()
# we define 3 Te for low, mid and high ionization regions
Tmid = tS3
mask_nan = np.isnan(tS3)
Tmid[mask_nan] = tO3[mask_nan] * 0.83 + 1700
    
Thigh = tO3
mask_nan = np.isnan(tO3)
Thigh[mask_nan] = (tS3[mask_nan] - 1700) / 0.83
    
#Tlow = (temp_S2+temp_O2) / 2.    
Tlow = Thigh*0.7 + 3000

print(Tlow, Tmid, Thigh)

### Compute the ionic abundances for N, O, Ne for all the objects.

In [ ]:
#we associate the Te with each ion
Te_dic = {'N2' : Tlow,
              'O2' : Tlow,
              'S2' : Tlow,
              'S3' : Tmid,
              'Ar3' : Tmid,
              'O3'  : Thigh,
              'Ne3' : Thigh}
    
all_atoms = pn.getAtomDict(atom_list=obs.getUniqueAtoms())
    
# This is the dictionnary which will contain the ionic abundances
ab_dic = {}
    
# we  use the following lines to determine the ionic abundances
ab_labels = ['N2_6584A', 'O2_3727A+', 'O3_5007A', 'S2_6716A', 'S3_9069A', 'Ar3_7136A', 'Ne3_3869A']
for line in obs.getSortedLines():
    if line.label in ab_labels:
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, Te_dic[line.atom], mean_dens, 
                                                   to_eval=line.to_eval, Hbeta=100)
        print('{0:9s}'.format(line.label) + ' '.join(['{0:>4.2f}'.format(t) for t in 12 + np.log10(ab)]))
        ab_dic[line.atom] = ab

### Compute the total abundances of N, O, Ne for all the objects using the ’classical’ formulae for the ionization correction factors: O/H= O$^+$/H$^+$ + O$^{++}$/H$^+$, N/O = N$^+$/O$^+$, and Ne/O=Ne$^{++}$ /O$^{++}$ .

##### We compute the total abundances by summing the ionic ones for O/H and by using classical ICFs.

In [ ]:
OoH = ab_dic['O2'] + ab_dic['O3']
NoH = ab_dic['N2'] * OoH / ab_dic['O2']
NeoH = ab_dic['Ne3'] * OoH / ab_dic['O3']

In [ ]:
print(OoH)

In [ ]:
print(12+np.log10(OoH))

### Plot O/H, Ne/H, N/H as a function of the galactocentric distance (given in Bresolin et al 2009).

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
ax.scatter(Rgal, np.log10(OoH) + 12, label = 'O/H', color='red')
ax.scatter(Rgal, np.log10(NoH) + 12, label = 'N/H', color='blue')
ax.scatter(Rgal, np.log10(NeoH) + 12, label = 'Ne/H', color='green')

ax.legend()
ax.set_xlabel('RGal')
ax.set_ylabel('12+log(X/H)')

### Print out all the atomic data files used in this programm.

In [ ]:
for ion in np.sort(list(pn.config.DataFiles.keys())):
    print('{0} -> {1}'.format(ion, pn.config.DataFiles[ion]))

### ??? How should one proceed (in principle) to compute properly the error bars?

In [1]:
# the following is to have the nice style in the Notebook.
# Don't remove this.
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()